In [1]:
import pandas as pd
import numpy as np
import glob, os
import datetime
import json

In [2]:
pd.set_option('display.max_columns', None)  
# pd.set_option('display.max_colwidth', 125)  # or 199

Making some functions below for preprocessing / cleaning.
 - Each monthly file is zipped, and once unzipped, makes up 50+ csv files. **build_dataframes** merges them all together in one dataframe. So there will be one dataframe per monthly data pull. I then reset_index() so that the index does not have duplicates
 - the date and time information is in unix time, so have to convert that. Then create a couple more columns, **time_to_launch**, and **day_limit** using these new columns (**create_date_cols**)
 - There are a few columns that are in json format (I believe original data is NoSQL with nested collections). I first have to convert all "null" values to "none" so it can be read as a python dictionary (**replace_null**), and then convert each of these columns to a series of new columns, using the original column name as the prefix (**convert_json_columns**)
 
I will eventually create a python script for this, and then import it here

In [3]:
def build_dataframe(path):
    '''
    this merges all files for each datapull into one dataframe
    '''
    all_files = glob.glob(os.path.join(path, "*.csv"))
    list_ = []
    for file_ in all_files:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
    return pd.concat((pd.read_csv(f) for f in all_files)).reset_index()

In [4]:
def create_date_cols(df):
    '''
    creates custom datetime columns, and time_to_launch, day_limit
    '''
    df['date_created'] = pd.to_datetime(df['created_at'], unit='s') # when project creator started work on project
    df['deadline_date'] = pd.to_datetime(df['deadline'], unit='s')
    df['launched_date'] = pd.to_datetime(df['launched_at'], unit='s') # when creator launched project, starts countdown for deadline
    df['time_to_launch'] = (df['launched_date'] - df['date_created']).dt.days # num days from creation to launch
    df['day_limit'] = (df['deadline_date'] - df['launched_date']).dt.days # num days from launch to deadline
    return df

In [5]:
def clean_creator(text):
    '''
    cleans the text within creator column. used in function 'clean_creator_col' to clean the entire column
    '''
    head, sep, tail = text.partition(',"name"')
    return head + '}'

In [6]:
def clean_creator_col(df):
    '''
    creates column that can then be processed with json.loads
    '''
    df['creator_cleaned'] = df['creator'].apply(clean_creator)
    return df

In [7]:
def clean_url(text):
    '''
    cleans the text within url column. used in function 'clean_url_col' to clean the entire column
    '''
    head, sep, tail = text.partition('"web":')
    text = tail
    return tail[:-1]
    

In [8]:
def clean_url_col(df):
    '''
    creates column that can then be processed with json.loads
    '''
    df['urls_cleaned'] = df['urls'].apply(clean_url)
    return df

In [9]:
def clean_profile(text):
    '''
    cleans the text within profile column. used in function 'clean_profile_col' to clean the entire column
    '''
    head, sep, tail = text.partition(',"state":')
    return head + '}'

In [10]:
def clean_profile_col(df):
    '''
    creates column that can then be processed with json.loads
    '''
    df['profile_cleaned'] = df['profile'].apply(clean_profile)
    return df

In [11]:
def clean_location(text):
    '''
    cleans the text within location column. used in function 'clean_location_col' to clean the entire column
    '''
    head, sep, tail = text.partition(',"slug":')
    return head + '}'

In [12]:
def clean_location_col(df):
    '''
    creates column that can then be processed with json.loads
    '''
    df['location_cleaned'] = df['location'].apply(clean_location)
    return df

In [13]:
def convert_json_columns(df):
    '''
    converts columns with json data to separate columns, using original column name as prefix
    '''
    
    #category column **WORKING**
    cat_df = df['category'].apply(json.loads)
    cat_df = pd.DataFrame(cat_df.tolist()).add_prefix('category_')
    df = pd.merge(df, cat_df, left_index=True, right_index=True, how='outer')
    
    #creator column **WORKING**
    creator_df = df['creator_cleaned'].apply(json.loads)
    creator_df = pd.DataFrame(creator_df.tolist()).add_prefix('creator_')
    df = pd.merge(df, creator_df, left_index=True, right_index=True, how='outer')
    
#     #location column **NOT WORKING - I think it's due to null values, but this data isn't important**
#     location_df = df['location_cleaned'].apply(json.loads)
#     location_df = pd.DataFrame(location_df.tolist()).add_prefix('location_')
#     df = pd.merge(df, location_df, left_index=True, right_index=True, how='outer')
    
    #profile column **WORKING**
    profile_df = df['profile_cleaned'].apply(json.loads)
    profile_df = pd.DataFrame(profile_df.tolist()).add_prefix('profile_')
    df = pd.merge(df, profile_df, left_index=True, right_index=True, how='outer')
    
    #urls column **WORKING (but need to clean a bit first like with creator)**
    urls_df = df['urls_cleaned'].apply(json.loads)
    urls_df = pd.DataFrame(urls_df.tolist()).add_prefix('urls_')
    df = pd.merge(df, urls_df, left_index=True, right_index=True, how='outer')
    
    return df

Below are my paths to my files for each data pull:

In [14]:
path_jan2019 =r'/Users/robertpagano/Desktop/Kickstarter Data/Kickstarter_2019-01-17'
path_dec2018 =r'/Users/robertpagano/Desktop/Kickstarter Data/Kickstarter_2018-12-13'

In [15]:
df_jan2019 = build_dataframe(path_jan2019)
# df_jan2019.head(5)

In [16]:
create_date_cols(df_jan2019)

,index,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,launched_at,location,name,permissions,photo,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,date_created,deadline_date,launched_date,time_to_launch,day_limit
0,0,1,An eco-friendly coffee table that is both func...,"{""id"":356,""name"":""Woodworking"",""slug"":""crafts/...",240,US,1473631551,"{""id"":983022919,""name"":""Rhys Kucharski"",""is_re...",USD,$,True,USD,1480966943,False,NaN,1.000000,5000.0,1504859185,NaN,False,NaN,1478371343,"{""id"":2357536,""name"":""Austin"",""slug"":""austin-t...",Industrial Bamboo Table,NaN,"{""key"":""assets/013/695/911/e0a26959b2f3a1dfc64...",240.00,"{""id"":2674851,""project_id"":2674851,""state"":""in...",industrial-bamboo-table,https://www.kickstarter.com/discover/categorie...,False,False,failed,1480966943,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",240.000000,international,2016-09-11 22:05:51,2016-12-05 19:42:23,2016-11-05 18:42:23,54,30
1,1,3,We take digital uploads and make them handpain...,"{""id"":23,""name"":""Painting"",""slug"":""art/paintin...",243,CA,1436540372,"{""id"":101531536,""name"":""Picadoo Custom Art"",""i...",CAD,$,True,USD,1440417634,False,NaN,0.753746,1000.0,49266114,NaN,False,NaN,1437480034,"{""id"":4063,""name"":""London"",""slug"":""london-ca"",...","Custom Pet Portraits on Canvas- ""From Pixels t...",NaN,"{""key"":""assets/012/193/266/007123f2ab3331c4d1b...",322.00,"{""id"":2007748,""project_id"":2007748,""state"":""in...",custom-pet-portraits-on-canvas-from-pixels-to-...,https://www.kickstarter.com/discover/categorie...,False,False,failed,1440417634,0.770032,"{""web"":{""project"":""https://www.kickstarter.com...",247.950175,domestic,2015-07-10 14:59:32,2015-08-24 12:00:34,2015-07-21 12:00:34,10,34
2,2,243,We are a team of restaurant pros looking to fu...,"{""id"":311,""name"":""Food Trucks"",""slug"":""food/fo...",41738,US,1427218874,"{""id"":372111659,""name"":""Kevin, Domingo & Suzie...",USD,$,True,USD,1431706954,False,NaN,1.000000,35000.0,1228074690,NaN,False,NaN,1429114954,"{""id"":2367105,""name"":""Boston"",""slug"":""boston-m...",The Barmobile: Boston's Mobile Cocktail Cateri...,NaN,"{""key"":""assets/012/081/761/cc38b3bba0cdd4ac01e...",41738.00,"{""id"":1793461,""project_id"":1793461,""state"":""in...",the-barmobile-bostons-mobile-cocktail-catering...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1431706954,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",41738.000000,international,2015-03-24 17:41:14,2015-05-15 16:22:34,2015-04-15 16:22:34,21,30
3,3,27,"Loosely-based on a Lakota legend, Grandfather ...","{""id"":46,""name"":""Children's Books"",""slug"":""pub...",3115,US,1495110632,"{""id"":1336552462,""name"":""Christine Almstrom"",""...",USD,$,True,USD,1500217383,False,NaN,1.000000,3000.0,330962986,NaN,False,NaN,1496329383,"{""id"":2413146,""name"":""Grantham"",""slug"":""granth...",Grandfather Thunder & The Night Horses,NaN,"{""key"":""assets/016/724/938/812b782f5cd1d67fa3e...",3115.00,"{""id"":3005247,""project_id"":3005247,""state"":""in...",grandfather-thunder-and-the-night-horses,https://www.kickstarter.com/discover/categorie...,True,False,successful,1500217384,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",3115.000000,domestic,2017-05-18 12:30:32,2017-07-16 15:03:03,2017-06-01 15:03:03,14,45
4,4,3,Save me is a feature film about a depression s...,"{""id"":298,""name"":""Movie Theaters"",""slug"":""film...",674,IE,1450121921,"{""id"":2032403632,""name"":""David Russell"",""is_re...",EUR,€,False,USD,1455328590,False,NaN,1.141043,15000.0,1657821447,NaN,False,NaN,1450144590,"{""id"":562503,""name"":""Wicklow"",""slug"":""wicklow-...",Save Me-A film to hightligh

In [17]:
df_jan2019 = clean_creator_col(df_jan2019)

In [18]:
df_jan2019 = clean_url_col(df_jan2019)

In [19]:
df_jan2019 = clean_profile_col(df_jan2019)

In [20]:
df_jan2019 = convert_json_columns(df_jan2019)

In [24]:
df_jan2019.tail()

,index,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,launched_at,location,name,permissions,photo,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,date_created,deadline_date,launched_date,time_to_launch,day_limit,creator_cleaned,urls_cleaned,profile_cleaned,category_color,category_id,category_name,category_parent_id,category_position,category_slug,category_urls,creator_id,profile_id,profile_project_id,urls_message_creator,urls_project,urls_rewards
207843,3822,86,1000+ pages for $5.00! Two highly acclaimed sp...,"{""id"":252,""name"":""Graphic Novels"",""slug"":""comi...",2293,AU,1491774581,"{""id"":188920423,""name"":""Shane W Smith"",""slug"":...",AUD,$,True,USD,1503574200,False,NaN,0.719603,1000.0,2075668219,NaN,False,NaN,1499639424,"{""id"":1100968,""name"":""Canberra"",""slug"":""canber...",Bringing The Lesser Evil & Peaceful Tomorrows ...,NaN,"{""key"":""assets/016/311/446/e0e470bdfa4408b3a44...",2902.66,"{""id"":2955483,""project_id"":2955483,""state"":""ac...",bringing-the-lesser-evil-and-peaceful-tomorrow...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1503574200,0.760453,"{""web"":{""project"":""https://www.kickstarter.com...",2207.337608,domestic,2017-04-09 21:49:41,2017-08-24 11:30:00,2017-07-09 22:30:24,91,45,"{""id"":188920423}","{""project"":""https://www.kickstarter.com/projec...","{""id"":2955483,""project_id"":2955483}",16776056,252,Graphic Novels,3.0,4,comics/graphic novels,{'web': {'discover': 'http://www.kickstarter.c...,188920423,2955483,2955483,NaN,https://www.kickstarter.com/projects/shanewsmi...,https://www.kickstarter.com/projects/shanewsmi...
207844,3823,20,I would like to purchase photo and video equip...,"{""id"":15,""name"":""Photography"",""slug"":""photogra...",627,US,1413673139,"{""id"":677126394,""name"":""Diana Bottomley"",""is_r...",USD,$,True,USD,1416291333,False,NaN,1.000000,575.0,458068137,NaN,False,NaN,1413695733,"{""id"":2443945,""name"":""Madison"",""slug"":""madison...",Cameras for Kids - An Early Childhood Experience,NaN,"{""key"":""assets/011/913/865/825070ab96ee3403013...",627.00,"{""id"":1450942,""project_id"":1450942,""state"":""in...",cameras-for-kids-an-early-childhood-experience,https://www.kickstarter.com/discover/categorie...,True,False,successful,1416291333,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",627.000000,domestic,2014-10-18 22:58:59,2014-11-18 06:15:33,2014-10-19 05:15:33,0,30,"{""id"":677126394}","{""project"":""https://www.kickstarter.com/projec...","{""id"":1450942,""project_id"":1450942}",58341,15,Photography,NaN,12,photography,{'web': {'discover': 'http://www.kickstarter.c...,677126394,1450942,1450942,NaN,https://www.kickstarter.com/projects/677126394...,https://www.kickstarter.com/projects/677126394...
207845,3824,24,Having a baby or looking for the perfect gift ...,"{""id"":341,""name"":""Wearables"",""slug"":""technolog...",1175,CA,1415048821,"{""id"":1451299208,""name"":""NapTime"",""slug"":""napt...",CAD,$,True,USD,1423333581,False,NaN,0.753746,100000.0,1138681061,NaN,False,NaN,1420741581,"{""id"":3534,""name"":""Montreal"",""slug"":""montreal-...",NapTime: the first baby monitor that takes car...,NaN,"{""key"":""assets/011/932/396/a43bcf67aff6a353127...",1471.00,"{""id"":1488992,""project_id"":1488992,""state"":""in...",naptime-a-silent-baby-monitor-that-takes-care-...,https://www.kickstarter.com/discover/categorie...,False,False,failed,1423333581,0.846146,"{""web"":{""project"":""https://www.kickstarter.com...",1244.680913,domestic,2014-11-03 21:07:01,2015-02-07 18:26:21,2015-01-08 18:26:21,65,30,"{""id"":1451299208}","{""project"":""https://www.kickstarter.com/projec...","{""id"":1488992,""project_id"":1488992}",6526716,341,Wearables,16.0,14,technology/w

In [23]:
df_jan2019.columns

Index(['index', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'launched_at', 'location', 'name', 'permissions', 'photo', 'pledged',
       'profile', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type', 'date_created', 'deadline_date', 'launched_date',
       'time_to_launch', 'day_limit', 'creator_cleaned', 'urls_cleaned',
       'profile_cleaned', 'category_color', 'category_id', 'category_name',
       'category_parent_id', 'category_position', 'category_slug',
       'category_urls', 'creator_id', 'profile_id', 'profile_project_id',
       'urls_message_creator', 'urls_project', 'urls_rewards'],
      dtype='ob

In [25]:
df_jan2019.describe()

,index,backers_count,converted_pledged_amount,created_at,deadline,fx_rate,goal,id,launched_at,pledged,state_changed_at,static_usd_rate,usd_pledged,time_to_launch,day_limit,category_color,category_id,category_parent_id,category_position,creator_id,profile_id,profile_project_id
count,207848.000000,207848.000000,2.078480e+05,2.078480e+05,2.078480e+05,207848.000000,2.078480e+05,2.078480e+05,2.078480e+05,2.078480e+05,2.078480e+05,207848.000000,2.078480e+05,207848.000000,207848.000000,2.078480e+05,207848.000000,198764.00000,207848.000000,2.078480e+05,2.078480e+05,2.078480e+05
mean,1904.391628,148.180016,1.330363e+04,1.453200e+09,1.460120e+09,0.993843,4.971114e+04,1.073122e+09,1.457296e+09,1.909559e+04,1.459956e+09,1.011900,1.318990e+04,46.915717,32.484104,1.184646e+07,178.025066,11.75884,6.909905,1.073982e+09,2.195338e+06,2.195338e+06
std,1105.787122,890.405307,9.018058e+04,6.192901e+07,6.157192e+07,0.205432,1.187729e+06,6.195257e+08,6.161166e+07,3.306954e+05,6.146450e+07,0.231238,8.931127e+04,129.938729,11.799842,6.067282e+06,134.201226,5.51711,4.540742,6.202986e+08,1.054993e+06,1.054993e+06
min,0.000000,0.000000,0.000000e+00,1.240366e+09,1.241334e+09,0.007127,1.000000e-02,8.624000e+03,1.240603e+09,0.000000e+00,1.241334e+09,0.008771,0.000000e+00,0.000000,1.000000,5.162700e+04,1.000000,1.00000,1.000000,3.000000e+00,1.900000e+01,1.900000e+01
25%,944.000000,4.000000,1.050000e+02,1.412086e+09,1.419048e+09,1.000000,1.500000e+03,5.349892e+08,1.416272e+09,1.060000e+02,1.418940e+09,1.000000,1.050000e+02,2.000000,29.000000,6.526716e+06,37.000000,9.00000,3.000000,5.366696e+08,1.399988e+06,1.399988e+06
50%,1900.000000,27.000000,1.546000e+03,1.455758e+09,1.462488e+09,1.000000,5.000000e+03,1.075613e+09,1.459693e+09,1.560000e+03,1.462399e+09,1.000000,1.547000e+03,11.000000,30.000000,1.486766e+07,252.000000,12.00000,6.000000,1.072812e+09,2.380312e+06,2.380312e+06
75%,2863.000000,89.000000,6.650000e+03,1.507387e+09,1.514011e+09,1.000000,1.500000e+04,1.608395e+09,1.511215e+09,7.003000e+03,1.513974e+09,1.000000,6.646000e+03,36.000000,34.000000,1.674378e+07,302.000000,16.00000,10.000000,1.611959e+09,3.173402e+06,3.173402e+06
max,3829.000000,105857.000000,8.596474e+06,1.547684e+09,1.552882e+09,1.799413,1.000000e+08,2.147476e+09,1.547701e+09,8.103074e+07,1.547701e+09,1.716408,8.596475e+06,3303.000000,93.000000,1.677606e+07,389.000000,26.00000,19.000000,2.147483e+09,3.644883e+06,3.644883e+06
